<a href="https://colab.research.google.com/github/Marine27/TER/blob/master/PatternMining/Process_for_structure_pm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mise en forme des donnees pour le *Pattern Mining*

### Imports

In [0]:
import pandas as pd  
import numpy as np
import warnings

### Recuperation et preparation des donnnees

In [0]:
#Chargement du jeu de donnees
data_child = pd.read_csv('/Users/marine/Desktop/Projet_TER/TER/data/data_madeleine_final.csv',
                           sep = '\t',
                           encoding = 'utf-8',
                           index_col = False)

# Supression de la colonne qui duplique l'index
data_child = data_child.drop(columns = 'Unnamed: 0') 

# Recuperation de la partie phonetique des intervention de l'enfant
data_child = data_child.where(data_child['type']=='pho').dropna(subset=['type'])

# Conservation des deux colonnes avec des informations pertinantes 
data_child = data_child[['age','contenu']]

# Creer un indexage a partir de cet ordre
data_child['index']=[ i for i in range(len(data_child))]
data_child= data_child.set_index('index')

# Creation d'un numpy utile par la suite pour les boucles
ages = data_child.age.unique()

# Indexage par l'age de l'enfant
age = data_child['age']

l=[i for i in range(len(age))]
k=[age.values]
k.append(l)
tuples = list(zip(*k))

index = pd.MultiIndex.from_tuples(tuples, names=['age', 'index'])
data_child.set_index(index,inplace=True)

# Supression de la colonne age car present en index
data_child=data_child[['contenu']]

data_child.head()

### Demarches pour la division de chaque enregistrement en 4 parties

In [0]:
# Sépare en mots au lieu de phrases
mots = {}
for i in list(ages):
    # recupere toutes les phrases a la i eme date
    phrases = data_child['contenu'].loc[i].values
    # creer une variable texte avec toutes les phrases
    phrase = ' '.join(phrases)
    # sépare en mot
    mots[i] = phrase.split()
    
# se parcourt avec un age en parametre
#mots["1_00_05"]

In [0]:
# Sépare en 4 "enfants" chaque enregistrement
enregistrement = {}
sizes = []
for i in list(ages):
    # recupere tout les mots a la i eme date
    #data = data_child['contenu'].loc[i].values
    data = mots[i]
    # separe les mots en 4 groupes ( 3 goupes de meme dim et le dernier avec le reste)
    if len(data)<4:
        warnings.warn("\nAttention un l'enregistrement "+ i + " contient moins de 4 mots")
    s = len(data)//4
    sizes.append(s)
    enfant1 = data[:s]
    enfant2 = data[s:2*s]
    enfant3 = data[2*s:3*s]
    enfant4 = data[3*s:]
    # creer une variable texte de chaque goupe
    #enfant1 = ' '.join(enfant1)
    #enfant2 = ' '.join(enfant2)
    #enfant3 = ' '.join(enfant3)
    #enfant4 = ' '.join(enfant4)
    
    enfant1 = tuple(enfant1)
    enfant2 = tuple(enfant2)
    enfant3 = tuple(enfant3)
    enfant4 = tuple(enfant4)
    
    # sépare en 4 enfants
    enregistrement[i] = [enfant1, enfant2, enfant3, enfant4]
    
# se parcourt avec un age en parametre
#enregistrement["1_00_05"]

In [0]:
# Pour avoir une idée du nombre de mots par enregistrement après répartition
pd.DataFrame(sizes[:-1]).describe().T

In [0]:
# Crée une DataFrame de toutes les donnees
tab_4groupes = []
#sizes = []
for i in list(ages):
    t = enregistrement[i]
    #s = len(t)
    tab_4groupes.append(t)
    #sizes.append(s)
    
tab_madeleine = pd.DataFrame(tab_4groupes)

etiquettes = ["madeleine_enfant1",
              "madeleine_enfant2", 
              "madeleine_enfant3", 
              "madeleine_enfant4"]

tab_madeleine.columns = etiquettes

tab_madeleine.head()
# tab_madeleine.loc[1] recherche par numero de l'enregistrement
# tab_madeleine[1] recherche par numero de l'enfant

### Addaptation de la structure

In [0]:
# transactions = (('a', 'b', 'c'), ('b'), ('a'), ('a', 'c', 'd'), ('b', 'c'), ('b', 'c'))
# doit etre de cette forme:
# madeleine_enfant1 = transactions
# un enregistrement = ('a', 'b', 'c')
# un mot = 'a'

In [0]:
madeleine_enfant1 = tuple(tab_madeleine["madeleine_enfant1"][:-1])
madeleine_enfant2 = tuple(tab_madeleine["madeleine_enfant2"][:-1])
madeleine_enfant3 = tuple(tab_madeleine["madeleine_enfant3"][:-1])
madeleine_enfant4 = tuple(tab_madeleine["madeleine_enfant4"][:])

In [0]:
print(type(madeleine_enfant1))
#print(madeleine_enfant1)

### PATTERN MINING

In [0]:
from pymining import itemmining
from pymining import assocrules

from pymining import seqmining

In [0]:
min_supp = 10
relim_input = itemmining.get_relim_input(madeleine_enfant1)
item_sets = itemmining.relim(relim_input, min_support=min_supp) # min_support= nbr au pif < lignes
item_sets

In [0]:
rules = assocrules.mine_assoc_rules(item_sets, min_support=min_supp, min_confidence=0.5)
rules

In [0]:
#seqs = ( 'caabc', 'abcb', 'cabc', 'abbca')
seqs = madeleine_enfant1

freq_seqs = seqmining.freq_seq_enum(seqs, 9)
for seq in freq_seqs:
    print (seq)